In [63]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import lxml

import time
from selenium import webdriver
from selenium.webdriver.common.by import By

import pandas as pd

# browser = webdriver.Safari()
# browser.get('https://trudvsem.ru/vacancy/search')
# time.sleep(10)
# browser.quit()

**ISSUES:**

- Не учтена логика сброса фильтров
- Изменить логику программы по скроллингу и клику на каждую вакансию

In [32]:
df = pd.DataFrame(columns=[
    'название',
    'компания',
    'регион',
    'зп',
    'дата обновы',
    'ссылка'
])

In [35]:
with webdriver.Chrome() as browser:
    
    # заход на сайт
    browser.get('https://trudvsem.ru/vacancy/search')
    browser.maximize_window()
    browser.delete_all_cookies()
    time.sleep(2)
    
    # убрать фильтры 
    browser.find_element(By.XPATH, '/html/body/header/div[2]/div/div[2]/div/button[1]').click()
    
    # ищем поле ввода и вводим текст
    inp = browser.find_element(By.XPATH, '/html/body/header/div[1]/div[1]/label[1]/input')
    inp.send_keys('системный аналитик')
    time.sleep(1)
    
    # # убираем фильтры из серч панели
    # search_panel = browser.find_element(By.XPATH, '/html/body/header/div[2]/div')
    # buttons = search_panel.find_elements(By.CLASS_NAME, 'search-panel__item')
    # for button in buttons:
    #     button = button.find_element(By.TAG_NAME, 'button')
    #     button.click()
    
    # ищем кнопку "найти" и жмем ёё
    browser.find_element(By.XPATH, '/html/body/header/div[1]/div[1]/button').click()
    time.sleep(1)
    
    button = browser.find_element(By.XPATH, '/html/body/main/div[2]/div[5]/div/div[1]/div[3]/div[11]/button')
    # скроллинг до конца
    try: 
        while True:
            browser.execute_script("return arguments[0].scrollIntoView(true);", button)
            time.sleep(1)
            button.click()
            time.sleep(1)
    except:
        pass
    
    header = browser.find_element(By.TAG_NAME, 'h1')
    browser.execute_script("return arguments[0].scrollIntoView(true);", header)
    
    # посчитать количество вакансий на странице
    vac_block_bar = browser.find_element(By.CLASS_NAME, 'main__search-sidebar')
    vac_block = vac_block_bar.find_elements(By.TAG_NAME, 'div')
    vac_block = [div for div in vac_block if 'search-results-simple-card mb-1' in str(div.get_attribute('class'))]
    print('Количество вакансий:', len(vac_block))
    print('Парсим вакансии:\n')
    
    # вывести информацию о вакансии
    for vac in vac_block:
        vac_info = {}
        vac_info['название'] = vac.find_element(By.TAG_NAME, 'strong').text
        titles = vac.find_elements(By.CLASS_NAME, r'content_small')
        vac_info['компания'] = titles[0].text
        vac_info['регион'] = titles[1].text
        vac_info['зп'] = vac.find_element(By.CLASS_NAME, r'search-results-simple-card__salary').text
        vac_info['дата обновы'] = vac.find_element(By.CLASS_NAME, r'date__preview').text.strip('Обновлено: ')
        vac_info['ссылка'] = vac.find_element(By.TAG_NAME, 'a').get_attribute('href')
        df.loc[len(df)] = vac_info

Количество вакансий: 38
Парсим вакансии:



In [45]:
df.iloc[0, -1]

'https://trudvsem.ru/vacancy/card/1197746421620/85531d80-3767-11ee-99f9-0d972cff014f'

In [69]:
ua = UserAgent().chrome
headers = {'User-Agent': ua}

r = requests.get(df.iloc[1, -1], timeout=5, headers=headers, allow_redirects=True)

soup = BeautifulSoup(r.content, 'lxml')

In [72]:
df.head()

,название,компания,регион,зп,дата обновы,ссылка
0,Системный аналитик,"ООО ""ЦИФРОМЕД""",г. Москва,до 150 000 руб.,28.09.2023,https://trudvsem.ru/vacancy/card/1197746421620...
1,Системный аналитик,VK,г. Москва,,28.09.2023,https://trudvsem.ru/vacancy/card/7226c750-02f1...
2,Системный аналитик,"ФИЛИАЛ ООО ""АГР""",г. Москва,от 100 000 руб.,26.09.2023,https://trudvsem.ru/vacancy/card/97f0f670-8a79...
3,Системный аналитик,"ПАО ""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК""",г. Москва,до 90 000 руб.,26.09.2023,https://trudvsem.ru/vacancy/card/1027739555282...
4,Системный аналитик,ООО «МУЛЬТИБОНУС»,г. Москва,от 150 000 руб.,25.09.2023,https://trudvsem.ru/vacancy/card/a2a56960-745e...


In [74]:
with webdriver.Chrome() as browser:
    
    # заход на сайт
    browser.get('https://trudvsem.ru/vacancy/search')
    browser.maximize_window()
    browser.delete_all_cookies()
    time.sleep(2)
    
    # убрать фильтры 
    browser.find_element(By.XPATH, '/html/body/header/div[2]/div/div[2]/div/button[1]').click()
    
    # ищем поле ввода и вводим текст
    inp = browser.find_element(By.XPATH, '/html/body/header/div[1]/div[1]/label[1]/input')
    inp.send_keys('data scientist')
    time.sleep(1)
    
    # # убираем фильтры из серч панели
    # search_panel = browser.find_element(By.XPATH, '/html/body/header/div[2]/div')
    # buttons = search_panel.find_elements(By.CLASS_NAME, 'search-panel__item')
    # for button in buttons:
    #     button = button.find_element(By.TAG_NAME, 'button')
    #     button.click()
    
    # ищем кнопку "найти" и жмем ёё
    browser.find_element(By.XPATH, '/html/body/header/div[1]/div[1]/button').click()
    time.sleep(1)
    
    
    # скроллинг до конца
    try: 
        button = browser.find_element(By.XPATH, '/html/body/main/div[2]/div[5]/div/div[1]/div[3]/div[11]/button')
        while True:
            browser.execute_script("return arguments[0].scrollIntoView(true);", button)
            time.sleep(1)
            button.click()
            time.sleep(1)
    except:
        pass
    
    header = browser.find_element(By.TAG_NAME, 'h1')
    browser.execute_script("return arguments[0].scrollIntoView(true);", header)
    
    # посчитать количество вакансий на странице
    vac_block_bar = browser.find_element(By.CLASS_NAME, 'main__search-sidebar')
    vac_block = vac_block_bar.find_elements(By.TAG_NAME, 'div')
    vac_block = [div for div in vac_block if 'search-results-simple-card mb-1' in str(div.get_attribute('class'))]
    print('Количество вакансий:', len(vac_block))
    print('Парсим вакансии:\n')
    
    # вывести информацию о вакансии
    for vac in vac_block:
        vac_info = {}
        vac_info['название'] = vac.find_element(By.TAG_NAME, 'strong').text
        titles = vac.find_elements(By.CLASS_NAME, r'content_small')
        vac_info['компания'] = titles[0].text
        vac_info['регион'] = titles[1].text
        vac_info['зп'] = vac.find_element(By.CLASS_NAME, r'search-results-simple-card__salary').text
        vac_info['дата обновы'] = vac.find_element(By.CLASS_NAME, r'date__preview').text.strip('Обновлено: ')
        vac_info['ссылка'] = vac.find_element(By.TAG_NAME, 'a').get_attribute('href')
        df.loc[len(df)] = vac_info

Количество вакансий: 6
Парсим вакансии:



In [75]:
df.tail()

,название,компания,регион,зп,дата обновы,ссылка
39,Middle/senior data scientist,СберМаркет,г. Москва,,13.09.2023,https://trudvsem.ru/vacancy/card/7226c750-02f1...
40,Lead data scientist (operations),СберМаркет,г. Москва,,13.09.2023,https://trudvsem.ru/vacancy/card/7226c750-02f1...
41,Разработчик-исследователь / data scientist в d...,VK,г. Москва,,04.09.2023,https://trudvsem.ru/vacancy/card/7226c750-02f1...
42,Data scientist/machine learning engineer в ком...,VK,г. Москва,,25.09.2023,https://trudvsem.ru/vacancy/card/7226c750-02f1...
43,Ml engineer / data scientist в команду автомат...,VK,г. Москва,,04.09.2023,https://trudvsem.ru/vacancy/card/7226c750-02f1...
